## Examples of types of interactions
In this note book we provide some examples of how to define local, nonlocal, single and multi-channel interactions.

In [1]:
import numpy as np

from jitr import reactions, rmatrix
from jitr.reactions.potentials import coulomb_charged_sphere, woods_saxon_potential
from jitr.utils import delta, kinematics

In [27]:
solver = rmatrix.Solver(nbasis=40)

In [28]:
# a local scalar interaction
def interaction_local(r, V0, W0, R0, a0, Zz):
    nuclear = woods_saxon_potential(r, V0, W0, R0, a0)
    coulomb = coulomb_charged_sphere(r, Zz, R0)
    return nuclear + coulomb

In [29]:
# a nonlocal scalar interaction
# note this exp(-(r-rp)**2) form is just made up for example purposes
# this is not the Perey-Buck form, which is actually l-dependent
def interaction_nonlocal(r, rp, V0, W0, R0, a0, Zz):
    nuclear = woods_saxon_potential(0.5 * (r + rp), V0, W0, R0, a0) * np.exp(
        (r - rp) ** 2
    )
    return nuclear

In [5]:
def interaction_2level(r, V0, W0, V1, W1, g, R0, a0, Zz):
    nuclear0 = woods_saxon_potential(r, V0, W0, R0, a0)
    nuclear1 = woods_saxon_potential(r, V1, W1, R0, a0)
    coupling = woods_saxon_potential(r, g, 0, R0, a0)
    coulomb = coulomb_charged_sphere(r, Zz, R0)
    return np.array([[nuclear0 + coulomb, coupling], [coupling, nuclear1 + coulomb]])

In [6]:
def interaction_2level_nonlocal(r, rp, V0, W0, V1, W1, g, R0, a0, Zz):
    nuclear0 = woods_saxon_potential(0.5 * (r + rp), V0, W0, R0, a0) * np.exp(-((r - rp) ** 2))
    nuclear1 = woods_saxon_potential(0.5 * (r + rp), V1, W1, R0, a0) * np.exp(-((r - rp) ** 2))
    coupling = woods_saxon_potential(0.5 * (r + rp), g, 0, R0, a0) * np.exp(-((r - rp) ** 2))
    return np.array([[nuclear0, coupling], [coupling, nuclear1]])

### Define the scattering system

In [7]:
Elab = 35  # MeV
Ca48 = (28, 20)
proton = (1, 1)

mass_target = kinematics.mass(*Ca48)
mass_projectile = kinematics.mass(*proton)
Zz = Ca48[1]*proton[1]

In [8]:
Ecm, mu, k, eta = kinematics.classical_kinematics(
    mass_target,
    mass_projectile,
    Elab,
    Zz,
)

In [9]:
# first the 1 level system
sys_1level = reactions.ProjectileTargetSystem(
    channel_radii=np.array([5 * np.pi]),
    l=np.array([0]),
    mass_target=mass_target,
    mass_projectile=mass_projectile,
    Ztarget=Ca48[1],
    Zproj=proton[1],
)
channels_1, asymptotics_1 = sys_1level.coupled(Ecm, mu, k, eta)

In [10]:
# now the 2-level system
sys_2level = reactions.ProjectileTargetSystem(
    nchannels=2,
    channel_radii=np.array([5 * np.pi, 5 * np.pi]),
    l=np.array([0, 0]),
    mass_target=mass_target,
    mass_projectile=mass_projectile,
    Ztarget=Ca48[1],
    Zproj=proton[1],
)
channels_2, asymptotics_2 = sys_2level.coupled(Ecm, mu, k, eta)

In [11]:
# grab some parameters out of a hat
V0 = 42.0
W0 = 18.1
R0 = 4.8
a0 = 0.7
V1 = V0
W1 = W0
g = 10

# the single channel interactions are equivalent other than the non-locality
params_local = (V0, W0, R0, a0, Zz)
params_nonlocal = params_local

# the 2-level interactions  are likewise equivalent other than the non-locality
params_2level = (V0, W0, V1, W1, g, R0, a0, Zz)
params_2level_nonlocal = params_2level

# the non-local potentials also have a local Coulomb term
params_coul = (Zz, R0)

### Let's compare the 1-level systems

In [12]:
R, S, uext_boundary = solver.solve(
    channels_1,
    asymptotics_1,
    local_interaction=interaction_local,
    local_args=params_local,
)
phase_shift, phase_attenuation = delta(S[0, 0])
print(f"phase shift: {phase_shift:1.3f} + i ({phase_attenuation:1.3f}) [degrees]")

/home/beyerk/Projects/jitr/src/jitr/rmatrix/rmatrix_solver.py:44: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(complex128, 1, 'C', False, aligned=True), Array(complex128, 2, 'A', False, aligned=True))
  R, Ainv = rmatrix_with_inverse(A, b, nchannels, nbasis, a)


phase shift: -70.751 + i (-3.439) [degrees]


In [13]:
R, S, uext_boundary = solver.solve(
    channels_1,
    asymptotics_1,
    local_interaction=coulomb_charged_sphere,
    local_args=params_coul,
    nonlocal_interaction=interaction_nonlocal,
    nonlocal_args=params_nonlocal,
)
phase_shift, phase_attenuation = delta(S[0, 0])
print(f"phase shift: {phase_shift:1.3f} + i ({phase_attenuation:1.3f}) [degrees]")

phase shift: -59.658 + i (0.342) [degrees]


### Let's compare the 2-level systems


In [14]:
R, S, uext_boundary = solver.solve(
    channels_2,
    asymptotics_2,
    local_interaction=interaction_2level,
    local_args=params_2level,
)
for i in range(2):
    for j in range(2):
        phase_shift, phase_attenuation = delta(S[i,j])
        print(f"({i},{j}): phase shift: {phase_shift} + i ({phase_attenuation}) [degrees]")

(0,0): phase shift: -70.75802598415781 + i (-3.3759626594414436) [degrees]
(0,1): phase shift: 64.57316875195677 + i (74.31671545012324) [degrees]
(1,0): phase shift: 64.57316875195684 + i (74.31671545012696) [degrees]
(1,1): phase shift: -70.75802598416077 + i (-3.3759626594407934) [degrees]


In [15]:
R, S, uext_boundary = solver.solve(
    channels_2,
    asymptotics_2,
    nonlocal_interaction=interaction_2level_nonlocal,
    nonlocal_args=params_2level_nonlocal,
)
for i in range(2):
    for j in range(2):
        phase_shift, phase_attenuation = delta(S[i,j])
        print(f"({i},{j}): phase shift: {phase_shift} + i ({phase_attenuation}) [degrees]")

(0,0): phase shift: -68.16580407941652 + i (-3.406987982401112) [degrees]
(0,1): phase shift: 67.53296168480372 + i (74.02205276011857) [degrees]
(1,0): phase shift: 67.5329616848037 + i (74.02205276011587) [degrees]
(1,1): phase shift: -68.16580407941427 + i (-3.4069879824014) [degrees]


## Let's compare speed

In [30]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_1,
    asymptotics_1,
    local_interaction=interaction_local,
    local_args=params_local,
)

371 µs ± 5.73 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [17]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_1,
    asymptotics_1,
    local_interaction=coulomb_charged_sphere,
    local_args=params_coul,
    nonlocal_interaction=interaction_nonlocal,
    nonlocal_args=params_nonlocal,
)

461 µs ± 14.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_2,
    asymptotics_2,
    local_interaction=interaction_2level,
    local_args=params_2level,
)

898 µs ± 46.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [16]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_2,
    asymptotics_2,
    nonlocal_interaction=interaction_2level_nonlocal,
    nonlocal_args=params_2level_nonlocal,
)

1.13 ms ± 200 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Let's see if `njit`-ing the interaction gives us any speed up.

In [34]:
# a local scalar interaction
from numba import njit

@njit
def interaction_local_njit(r, V0, W0, R0, a0, Zz):
    nuclear = woods_saxon_potential(r, V0, W0, R0, a0)
    return nuclear

In [35]:
# jit compile once 
solver.solve(
    channels_1,
    asymptotics_1,
    local_interaction=interaction_local,
    local_args=params_local,
)

(array([[-0.01205059-0.0040834j]]),
 array([[-0.88242801-0.7018889j]]),
 array([0.32840734-0.97451344j]))

In [36]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_1,
    asymptotics_1,
    local_interaction=interaction_local,
    local_args=params_local,
)

378 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Answer, not really worth it.